In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import math
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew, norm
from scipy.special import boxcox
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import warnings
import datetime
import time

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use(style='seaborn')

In [ ]:
train = pd.read_csv('../input/bike-sharing-demand/train.csv')
test = pd.read_csv('../input/bike-sharing-demand/test.csv')
sample = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample.head()

Columns Info
datetime - hourly date + timestamp
season - 1 = spring, 2 = summer, 3 = fall, 4 = winter
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
temp - temperature in Celsius
atemp - "feels like" temperature in Celsius
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
count - number of total rentals

Objective:
count of bikes rented during each hour

In [ ]:
train.describe()

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

In [ ]:
train['datetime']

In [ ]:
# Training set
train['hour'] = [t.hour for t in pd.DatetimeIndex(train['datetime'])]
train['day'] = [t.day for t in pd.DatetimeIndex(train['datetime'])]
train['month'] = [t.month for t in pd.DatetimeIndex(train['datetime'])]
train['year'] = [t.year for t in pd.DatetimeIndex(train['datetime'])]
train.head(10)

In [ ]:
# Testing set
test['hour'] = [t.hour for t in pd.DatetimeIndex(test['datetime'])]
test['day'] = [t.day for t in pd.DatetimeIndex(test['datetime'])]
test['month'] = [t.month for t in pd.DatetimeIndex(test['datetime'])]
test['year'] = [t.year for t in pd.DatetimeIndex(test['datetime'])]
test.head(10)

In [ ]:
# Get the test id
test_id = test['datetime']
test_id

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
# Drop extra columns at the training set
train.drop(['casual', 'registered'], axis=1, inplace=True)
print(train.shape)
print(test.shape)

In [ ]:
sns.boxplot(train['count'])

In [ ]:
sns.distplot(train['count'], fit=norm)

In [ ]:
train['count']

In [ ]:
train['count'] = np.log(train['count'])
train['count']

In [ ]:
sns.distplot(train['count'], fit=norm)

In [ ]:
cat_features = ['season', 'holiday', 'workingday', 'weather']
i = 1
for col in cat_features:
    plt.subplot(2, 2, i)
    i += 1
    sns.countplot(train[col])
plt.show()

In [ ]:
num_features = ['temp', 'atemp', 'humidity', 'windspeed']
i = 1
for col in num_features:
    plt.subplot(2, 2, i)
    i += 1
    sns.boxplot(train[col]) 

plt.show()

In [ ]:
corr = train.corr()
plt.subplots(figsize=(12, 12))
sns.heatmap(corr, annot=True)

In [ ]:
plt.scatter(train['count'], train['windspeed'])
plt.show()

In [ ]:
plt.scatter(train['count'], train['season'])
plt.show()

In [ ]:
sns.distplot(train['season'])

In [ ]:
train.skew()

In [ ]:
train.drop(['holiday'], axis=1, inplace=True)
test.drop(['holiday'], axis=1, inplace=True)

In [ ]:
target = train['count']
train.drop(['datetime', 'count'], axis=1, inplace=True)
test.drop(['datetime'], axis=1, inplace=True)
print(train.shape)
print(test.shape)

In [ ]:
target

In [ ]:
# XGBoost
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, random_state =7, nthread = -1)
model_xgb.fit(train, target)

In [ ]:
# y_pred = model_xgb.predict(np.floor(np.exp(test)))
y_pred = model_xgb.predict(test)
y_pred = np.exp(y_pred)
y_pred

In [ ]:
sub = pd.DataFrame()
sub['datetime'] = test_id
sub['count'] = y_pred
sub.to_csv('submission.csv', index=False)